In [68]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Imports
from __future__ import absolute_import
from __future__ import print_function
from __future__ import print_function
from __future__ import absolute_import
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad, Adam
from keras.utils import np_utils, generic_utils
from keras import backend as K
#from tensorflow.keras import backend as k
#from tensorflow.python.keras import backend as k
from keras.layers.core import Lambda
from six.moves import range
import numpy as np
import scipy as sp
#from keras import backend as K
import random
import scipy.io
import matplotlib.pyplot as plt
from keras.regularizers import l2
import math
import numpy as np
import sys
import random
import warnings
import pprint
from six.moves import range
import six
import time
import os
import threading
from tensorflow.python.framework import ops
try:
    import queue
except ImportError:
    import Queue as queue
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.core import Dense, Dropout, Activation, Flatten

#Importing 5' Data
# Imports and functions from 5' Model
#%matplotlib inline
from matplotlib.colors import LogNorm
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv1D, SpatialDropout1D, MaxPooling1D, MaxPooling2D,Input, Flatten
from keras import models
from keras import optimizers
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import time
import sys
import six

nuc_arr = ['A','C','G','T']
#Function for calculating modified probability of splicing at SD1
def prob_SD1 (sd1_freq, sd2_freq):
    if (sd1_freq==0 and sd2_freq==0):
        return 0.0
    else:
        return sd1_freq/(sd1_freq+sd2_freq)
#Function converting nucleotide sequence to numerical array with 4 channels
def seq_to_arr (seq):
    seq_len = len(seq)
    arr_rep = np.zeros((seq_len, len(nuc_arr)))
    for i in range(seq_len):
        arr_rep[i][nuc_arr.index(seq[i])] = 1
    return arr_rep

#Creating a modified dataset with only the necessary information
#Storing model inputs and outputs
reads_path = "/content/gdrive/My Drive/Active Learning Research 2019/GSM1911086_A5SS_spliced_reads.txt"
seq_path = "/content/gdrive/My Drive/Active Learning Research 2019/GSM1911085_A5SS_seq.txt"
s1_indx = 1
s2_indx= 45
seq_len = 101
read_lines = []
seq_lines = []
data_table = []
with open(reads_path) as f:
    f.readline()
    for line in f:
        mod_line = line.split('\t')
        read_lines.append([mod_line[0], mod_line[s1_indx], mod_line[s2_indx]])
with open(seq_path) as f:
    f.readline()
    for line in f:
        mod_line = line.split('\t')
        seq_lines.append([mod_line[0], mod_line[1][:-1]])

n = len(read_lines)
prob_s1 = np.zeros(n)
inputs = np.zeros((n,seq_len, 4))

for i in range(n):
    prob_s1[i] = prob_SD1(float(read_lines[i][1]), float(read_lines[i][2]))
    inputs[i] = seq_to_arr(seq_lines[i][1])



In [70]:
def permanent_dropout_layer(rate):
    seed = np.random.randint(0,100)
    return Lambda(lambda x: K.dropout(x,level=rate,seed=seed))
"""
def standardize_X(X):
    if type(X) == list:
        return X
    else:
        return [X]

def permanent_spatial_dropout_layer(rate, noiseshape):
    seed = np.random.randint(0,100)
    return Lambda(lambda x: K.dropout(x,level=rate,noise_shape=noiseshape,seed=seed))
    

def predict_stochastic(model, X, batch_size=128, verbose=0):
    '''Generate output predictions for the input samples
    batch by batch.
    # Arguments
        X: the input data, as a numpy array.
        batch_size: integer.
        verbose: verbosity mode, 0 or 1.f
    # Returns
        A numpy array of predictions.
    '''
    X = standardize_X(X)
    pred_Y = model.predict(X,batch_size=batch_size, verbose=verbose)
    return np.array(pred_Y)
"""

"\ndef standardize_X(X):\n    if type(X) == list:\n        return X\n    else:\n        return [X]\n\ndef permanent_spatial_dropout_layer(rate, noiseshape):\n    seed = np.random.randint(0,100)\n    return Lambda(lambda x: K.dropout(x,level=rate,noise_shape=noiseshape,seed=seed))\n    \n\ndef predict_stochastic(model, X, batch_size=128, verbose=0):\n    '''Generate output predictions for the input samples\n    batch by batch.\n    # Arguments\n        X: the input data, as a numpy array.\n        batch_size: integer.\n        verbose: verbosity mode, 0 or 1.f\n    # Returns\n        A numpy array of predictions.\n    '''\n    X = standardize_X(X)\n    pred_Y = model.predict(X,batch_size=batch_size, verbose=verbose)\n    return np.array(pred_Y)\n"

In [0]:
np.random.seed(1)

Queries = 1

all_rmse_rand = 0
all_rmse_bald = 0
batch_size = 256
dropout_iterations = 100
size_train = 768
X = np.array(inputs)
y = np.array(prob_s1)
all_train_indices = np.load('/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/trainindices1.npy')
train_data_indices = all_train_indices[0:size_train].tolist()
test_indices = np.load('/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/testindices1.npy')
X_train = X[train_data_indices]
y_train = y[train_data_indices]
X_test = X[test_indices]
y_test = y[test_indices]

#X_pool = X[ index_pool, :  ]
#y_pool = y[ index_pool ]
#normalise the dataset - X_train, y_train and X_test
std_X_train = np.std(X_train, 0)
std_X_train[ std_X_train == 0  ] = 1
mean_X_train = np.mean(X_train, 0)
mean_y_train = np.mean(y_train)
std_y_train = np.std(y_train)

In [0]:
import keras.backend as K
from keras import initializers
from keras.engine import InputSpec
from keras.layers import Dense, Lambda, Wrapper

#From Concrete Dropout Paper, on GitHub
class ConcreteDropout(Wrapper):
    """This wrapper allows to learn the dropout probability for any given input Dense layer.
    ```python
        # as the first layer in a model
        model = Sequential()
        model.add(ConcreteDropout(Dense(8), input_shape=(16)))
        # now model.output_shape == (None, 8)
        # subsequent layers: no need for input_shape
        model.add(ConcreteDropout(Dense(32)))
        # now model.output_shape == (None, 32)
    
    `ConcreteDropout` can be used with arbitrary layers which have 2D
    kernels, not just `Dense`. However, Conv2D layers require different
    weighing of the regulariser (use SpatialConcreteDropout instead).
    # Arguments
        layer: a layer instance.
        weight_regularizer:
            A positive number which satisfies
                $weight_regularizer = l**2 / (\tau * N)$
            with prior lengthscale l, model precision $\tau$ (inverse observation noise),
            and N the number of instances in the dataset.
            Note that kernel_regularizer is not needed.
        dropout_regularizer:
            A positive number which satisfies
                $dropout_regularizer = 2 / (\tau * N)$
            with model precision $\tau$ (inverse observation noise) and N the number of
            instances in the dataset.
            Note the relation between dropout_regularizer and weight_regularizer:
                $weight_regularizer / dropout_regularizer = l**2 / 2$
            with prior lengthscale l. Note also that the factor of two should be
            ignored for cross-entropy loss, and used only for the eculedian loss.
    """

    def __init__(self, layer, weight_regularizer=1e-6, dropout_regularizer=1e-5,
                 init_min=0.1, init_max=0.1, is_mc_dropout=True, **kwargs):
        assert 'kernel_regularizer' not in kwargs
        super(ConcreteDropout, self).__init__(layer, **kwargs)
        self.weight_regularizer = weight_regularizer
        self.dropout_regularizer = dropout_regularizer
        self.is_mc_dropout = is_mc_dropout
        self.supports_masking = True
        self.p_logit = None
        self.p = None
        self.init_min = np.log(init_min) - np.log(1. - init_min)
        self.init_max = np.log(init_max) - np.log(1. - init_max)

    def build(self, input_shape=None):
        self.input_spec = InputSpec(shape=input_shape)
        if not self.layer.built:
            self.layer.build(input_shape)
            self.layer.built = True
        super(ConcreteDropout, self).build()  # this is very weird.. we must call super before we add new losses

        # initialise p
        self.p_logit = self.layer.add_weight(name='p_logit',
                                            shape=(1,),
                                            initializer=initializers.RandomUniform(self.init_min, self.init_max),
                                            trainable=True)
        self.p = K.sigmoid(self.p_logit[0])

        # initialise regulariser / prior KL term
        assert len(input_shape) == 2, 'this wrapper only supports Dense layers'
        input_dim = np.prod(input_shape[-1])  # we drop only last dim
        weight = self.layer.kernel
        kernel_regularizer = self.weight_regularizer * K.sum(K.square(weight)) / (1. - self.p)
        dropout_regularizer = self.p * K.log(self.p)
        dropout_regularizer += (1. - self.p) * K.log(1. - self.p)
        dropout_regularizer *= self.dropout_regularizer * input_dim
        regularizer = K.sum(kernel_regularizer + dropout_regularizer)
        self.layer.add_loss(regularizer)

    def compute_output_shape(self, input_shape):
        return self.layer.compute_output_shape(input_shape)

    def concrete_dropout(self, x):
        '''
        Concrete dropout - used at training time (gradients can be propagated)
        :param x: input
        :return:  approx. dropped out input
        '''
        eps = K.cast_to_floatx(K.epsilon())
        temp = 0.1

        unif_noise = K.random_uniform(shape=K.shape(x))
        drop_prob = (
            K.log(self.p + eps)
            - K.log(1. - self.p + eps)
            + K.log(unif_noise + eps)
            - K.log(1. - unif_noise + eps)
        )
        drop_prob = K.sigmoid(drop_prob / temp)
        random_tensor = 1. - drop_prob

        retain_prob = 1. - self.p
        x *= random_tensor
        x /= retain_prob
        return x

    def call(self, inputs, training=None):
        if self.is_mc_dropout:
            return self.layer.call(self.concrete_dropout(inputs))
        else:
            def relaxed_dropped_inputs():
                return self.layer.call(self.concrete_dropout(inputs))
            return K.in_train_phase(relaxed_dropped_inputs,
                                    self.layer.call(inputs),
                                    training=training)


In [76]:

#Original model, without spatial or concrete dropout

tau = 0.159708
N = X_train.shape[0]
dropout = 0.05
lengthscale = 1e2
Weight_Decay = (1 - dropout) / (2. * N * lengthscale**2 * tau)
model = Sequential()
model.add(Conv1D(seq_len,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(permanent_dropout_layer(dropout))
model.add(Conv1D(seq_len//2, (4), strides=1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(permanent_dropout_layer(dropout))
model.add(Dense(50, W_regularizer=l2(Weight_Decay), init='normal',  activation='relu'))
model.add(permanent_dropout_layer(dropout))
model.add(Dense(1, W_regularizer=l2(Weight_Decay), init='normal'))


model.compile(loss='mean_squared_error', optimizer='rmsprop')

#for layer in model.layers:
#    print(layer.get_output_at(0).get_shape().as_list())

model.fit(X_train, y_train, epochs=50, batch_size=batch_size, verbose=0)
All_Dropout_Scores = np.zeros(shape=(X_test.shape[0],1))
y_predicted = np.zeros(shape=(All_Dropout_Scores.shape[0]))
for d in range(dropout_iterations):
    dropout_score = predict_stochastic(model, X_test,batch_size=batch_size, verbose=0)
    All_Dropout_Scores = np.append(All_Dropout_Scores,dropout_score,axis=1)
for j in range(All_Dropout_Scores.shape[0]):
    P_temp = All_Dropout_Scores[j,:]
    P = np.delete(P_temp,0)
    P_mean = np.mean(P)
    y_predicted[j] = P_mean
mse = np.mean((y_test - y_predicted)**2)
print('r2 of test predictions: ' + str(r2_score(y_test, y_predicted)))
print('Mean Squared Error: ' + str(mse))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


r2 of test predictions: 0.4677616846996213
Mean Squared Error: 0.07593692326508099


In [74]:
#Version with Spatial Dropout after Conv Layers, but no Concrete Dropout

import tensorflow as tf 
from keras.models import Model

tau = 0.159708
batchsize = 256
N = X_train.shape[0]
dropout = 0.1
lengthscale = 1e2
Weight_Decay = (1 - dropout) / (2. * N * lengthscale**2 * tau)
inputlayer = Input(shape=(seq_len, len(nuc_arr)))
x = Conv1D(seq_len,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu')(inputlayer)
x = SpatialDropout1D(dropout)(x)
x = MaxPooling1D(pool_size=3)(x)
x = permanent_dropout_layer(dropout)(x)
x = Conv1D(seq_len//2, (4), strides=1, activation='relu')(x)
x = SpatialDropout1D(dropout)(x)
x = MaxPooling1D(pool_size=3)(x)
x = Flatten()(x)
x = permanent_dropout_layer(dropout)(x)
#x = Dense(50, activation='relu', kernel_initializer='random_normal', kernel_regularizer=keras.regularizers.l2(Weight_Decay))(x)
x = Dense(50, W_regularizer=l2(Weight_Decay), init='normal',  activation='relu')(x)
x = permanent_dropout_layer(dropout)(x)
#outputlayer = Dense(1, kernel_initializer='random_normal',kernel_regularizer = keras.regularizers.l2(Weight_Decay))(x)
outputlayer = Dense(1, W_regularizer=l2(Weight_Decay), init='normal')(x)

model = Model(inputlayer, outputlayer)


# for some model with dropout ...
f = K.function([model.layers[0].input, K.learning_phase()],
               [model.layers[-1].output])

def predict_with_uncertainty(f, x, n_iter=100):
    result = np.zeros((n_iter,x.shape[0]))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])[:,0]
        result[i,:] = predictions
    prediction = result.mean(axis=0)
    uncertainty = result.std(axis=0)
    return prediction, uncertainty    

model.compile(loss='mean_squared_error', optimizer='rmsprop')

model.fit(X_train, y_train, epochs=50, batch_size=batchsize, verbose=0)
predictions_with_uncertainty = predict_with_uncertainty(f, X_test, n_iter=100)
y_predicted = predictions_with_uncertainty[0]
y_uncertainty = predictions_with_uncertainty[1]
mse = np.mean((y_test - y_predicted)**2)
print('r2 of test predictions: ' + str(r2_score(y_test, y_predicted)))
print('Mean Squared Error: ' + str(mse))
print('uncertainties (std): ' + str(y_uncertainty))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


r2 of test predictions: 0.320351977765689
Mean Squared Error: 0.09696855380008416
uncertainties (std): [0.11393899 0.12971005 0.07174659 ... 0.11120183 0.04560007 0.02606147]


In [75]:
#Version with Spatial and Concrete Dropout

import tensorflow as tf 
from keras.models import Model

tau = 0.159708
batchsize = 128
N = X_train.shape[0]
dropout = 0.1
lengthscale = 1e2
Weight_Decay = (1 - dropout) / (2. * N * lengthscale**2 * tau)
inputlayer = Input(shape=(seq_len, len(nuc_arr)))
x = Conv1D(seq_len,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu')(inputlayer)
x = SpatialDropout1D(dropout)(x)
x = MaxPooling1D(pool_size=3)(x)
x = permanent_dropout_layer(dropout)(x)
x = Conv1D(seq_len//2, (4), strides=1, activation='relu')(x)
x = SpatialDropout1D(dropout)(x)
x = MaxPooling1D(pool_size=3)(x)
x = Flatten()(x)
x = permanent_dropout_layer(dropout)(x)
x = ConcreteDropout(Dense(50, W_regularizer=l2(Weight_Decay), init='normal',  activation='relu'), input_shape=(450,))(x)
outputlayer = ConcreteDropout(Dense(1, W_regularizer=l2(Weight_Decay), init='normal'), input_shape=(50,))(x)
model = Model(inputlayer, outputlayer)

# for some model with dropout ...
f = K.function([model.layers[0].input, K.learning_phase()],
               [model.layers[-1].output])

def predict_with_uncertainty(f, x, n_iter=100):
    result = np.zeros((n_iter,x.shape[0]))

    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])[:,0]
        result[i,:] = predictions
    prediction = result.mean(axis=0)
    uncertainty = result.std(axis=0)
    return prediction, uncertainty    

model.compile(loss='mean_squared_error', optimizer='rmsprop')

model.fit(X_train, y_train, epochs=50, batch_size=batchsize, verbose=0)
predictions_with_uncertainty = predict_with_uncertainty(f, X_test, n_iter=100)
y_predicted = predictions_with_uncertainty[0]
y_uncertainty = predictions_with_uncertainty[1]
mse = np.mean((y_test - y_predicted)**2)
print('r2 of test predictions: ' + str(r2_score(y_test, y_predicted)))
print('Mean Squared Error: ' + str(mse))
print('uncertainties (std): ' + str(y_uncertainty))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


r2 of test predictions: 0.4786600685352127
Mean Squared Error: 0.07438200000374148
uncertainties (std): [0.19067245 0.14921673 0.12157874 ... 0.13805234 0.09350427 0.032944  ]
